In [2]:
from planet import api
from planet.api import filters
from sys import stdout
import urllib.request
import shutil
from requests.auth import HTTPBasicAuth
import os
import time
import requests
import cv2
import glob
import subprocess
import rasterio
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import wikipedia
from xml.dom import minidom
import sys

In [11]:
title = input("Please input your volcano of interest: ")

Please input your volcano of interest: Anak Krakatoa


In [12]:
page = wikipedia.WikipediaPage(title)

In [13]:
print("Wikipedia summary for your volcano of interest: ")
print("")
print(wikipedia.summary(title))

Wikipedia summary for your volcano of interest: 

Anak Krakatoa (English: Child of Krakatoa) is an island in a caldera in the Sunda Strait between the islands of Java and Sumatra in the Indonesian province of Lampung. On December 29, 1927, Anak Krakatoa emerged from the caldera formed in 1883 by the explosive volcanic eruption that destroyed the island of Krakatoa. There has been sporadic eruptive activity at the site since the late 20th century, culminating with a large underwater collapse of the volcano which caused a deadly tsunami in December 2018, followed by subsequent activity in 2019 and an eruption in April 2020. Due to its young age, the island is one of several in the area which are of considerable interest to volcanologists, and the subject of extensive study.




In [14]:
print("References from the wikipedia article of your volcano of interest: ")
print("")

References_Wikipedia = page.references

len(References_Wikipedia)
x = 1

for i in References_Wikipedia:
    print('[',x,'] :',i)
    x = x + 1

References from the wikipedia article of your volcano of interest: 

[ 1 ] : http://in.reuters.com/article/worldNews/idINIndia-30436520071111?sp=true
[ 2 ] : http://portal.vsi.esdm.go.id/joomla/
[ 3 ] : http://doi.org/10.1144%2FSP361.7
[ 4 ] : http://sp.lyellcollection.org/content/361/1/79
[ 5 ] : http://tools.wmflabs.org/geohack/geohack.php?pagename=Anak_Krakatoa&params=6.102_S_105.423_E_type:mountain_scale:100000
[ 6 ] : http://www.worldcat.org/issn/0305-8719
[ 7 ] : https://www.bbc.com/news/science-environment-46707731
[ 8 ] : https://www.nbcnews.com/news/world/indonesia-s-anak-krakatau-volcano-triggered-deadly-tsunami-now-quarter-n952906
[ 9 ] : https://www.rt.com/news/441477-tour-boat-fire-bombed-volcano/
[ 10 ] : https://www.volcanodiscovery.com/krakatau/news/80657/Krakatau-volcano-Indonesia-activity-update-and-field-report-increasing-unrest.html
[ 11 ] : https://www.volcanodiscovery.com/krakatoa/2018/dec/eruption-tsunami/updates.html
[ 12 ] : https://volcano.si.edu/volcano.cfm?v

In [15]:
print("Related wikipedia articles to your volcano of interest: ")
print("")

Related_Wikipedia = wikipedia.geosearch(page.coordinates[0], page.coordinates[1])

len(Related_Wikipedia)
x = 1

for i in Related_Wikipedia:
    print ('[',x,'] :', i, '-', wikipedia.page(i).url)
    x = x + 1

Related wikipedia articles to your volcano of interest: 

[ 1 ] : Krakatoa archipelago - https://en.wikipedia.org/wiki/Krakatoa_archipelago
[ 2 ] : Anak Krakatoa - https://en.wikipedia.org/wiki/Anak_Krakatoa
[ 3 ] : Krakatoa - https://en.wikipedia.org/wiki/Krakatoa
[ 4 ] : 1883 eruption of Krakatoa - https://en.wikipedia.org/wiki/1883_eruption_of_Krakatoa
[ 5 ] : 2018 Sunda Strait tsunami - https://en.wikipedia.org/wiki/2018_Sunda_Strait_tsunami


In [16]:
lat = float(page.coordinates[0])
long = float(page.coordinates[1])

In [17]:
start_date = input("Please input your start date (YYYY-MM-DD): ")

Please input your start date (YYYY-MM-DD): 2018-12-15


In [18]:
end_date = input("Please input your end date (YYYY-MM-DD): ")

Please input your end date (YYYY-MM-DD): 2018-12-18


In [19]:
cloud_cover = input("Please input the percent of maximum cloud cover you want to allow in each image as an integer between 0 and 1 (e.g. 20% = 0.2): ")
cloud_cover = float(cloud_cover)

Please input the percent of maximum cloud cover you want to allow in each image as an integer between 0 and 1 (e.g. 20% = 0.2): 0.2


In [20]:
api_key = input("Please input you Planet API key: ") 

Please input you Planet API key: 219bc2e637384268a2d606dd8c05e7a7


In [21]:
client = api.ClientV1(api_key) 
PL_API_KEY = api_key

In [22]:
geo_json_geometry = {
  "type": "Polygon",
        "coordinates": [
          [
            [
              long - 0.07,
              lat + 0.07
            ],
            [
              long + 0.07,
              lat + 0.07
            ],
            [
              long + 0.07,
              lat - 0.07
            ],
            [
              long - 0.07,
              lat - 0.07
            ],
            [
              long - 0.07,
              lat + 0.07
            ]
          ]
        ]
      }

In [23]:
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geo_json_geometry
}

In [24]:
date_range_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config": {
        "gte": start_date + "T00:00:00.000Z",
        "lte": end_date + "T00:00:00.000Z"
    }
}

In [25]:
cloud_cover_filter = {
    "type": "RangeFilter",
    "field_name": "cloud_cover",
    "config": {
        "lte": cloud_cover,
    }
}

In [26]:
item_types = ['PSScene4Band']
asset_type = 'analytic'

In [27]:
query = {
    "type": "AndFilter",
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [28]:
request = filters.build_search_request(
    query, item_types
)

In [29]:
dataset = client.quick_search(request)

stdout.write('id,cloud_cover,date\n')

id,cloud_cover,date


In [30]:
for item in dataset.items_iter(limit=None):
    props = item['properties']
    stdout.write('{0},{cloud_cover},{acquired}\n'.format(item['id'], **props))

20181217_025142_101e,0.01,2018-12-17T02:51:42.296592Z
20181217_025144_101e,0.01,2018-12-17T02:51:44.363859Z
20181217_025143_101e,0.01,2018-12-17T02:51:43.330487Z


In [31]:
for i in item_types:
    request = api.filters.build_search_request(query, [i])  # ,name = None, interval= 'day')
    dataset = client.quick_search(request)
    print(i)
    for item in dataset.items_iter(limit=50):
        print(item['id'])
        session = requests.Session()
        session.auth = (PL_API_KEY, '')
        results = \
            session.get(
                ("https://api.planet.com/data/v1/item-types/" +
                 "{}/items/{}/assets/").format(i, item['id']))
       
        item_activation_url = results.json()[asset_type]["_links"]["activate"]
       
        response = session.post(item_activation_url)
        print(response.status_code)
        while response.status_code != 204:
            time.sleep(30)
            response = session.post(item_activation_url)
            response.status_code = response.status_code
            print(response.status_code)
        
        item_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(i, item['id'])
        result = requests.get(item_url, auth=HTTPBasicAuth(PL_API_KEY, ''))
        download_url = result.json()[asset_type]['location']  # KeyError: 'location'
        output_file = item['id'] + '.tif'

        with urllib.request.urlopen(download_url) as response, open(output_file, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)

PSScene4Band
20181217_025142_101e
204
20181217_025144_101e
204
20181217_025143_101e
204


In [32]:
item_types2 = ['PSScene4Band']
asset_type2 = 'analytic_xml'

In [33]:
query = {
    "type": "AndFilter",
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [34]:
request = filters.build_search_request(
    query, item_types2
)

In [35]:
dataset = client.quick_search(request)

stdout.write('id,cloud_cover,date\n')

id,cloud_cover,date


In [36]:
for item in dataset.items_iter(limit=None):
    props = item['properties']
    stdout.write('{0},{cloud_cover},{acquired}\n'.format(item['id'], **props))

20181217_025142_101e,0.01,2018-12-17T02:51:42.296592Z
20181217_025144_101e,0.01,2018-12-17T02:51:44.363859Z
20181217_025143_101e,0.01,2018-12-17T02:51:43.330487Z


In [37]:
for i in item_types2:
    request = api.filters.build_search_request(query, [i])  # ,name = None, interval= 'day')
    dataset = client.quick_search(request)
    print(i)
    for item in dataset.items_iter(limit=50):
        print(item['id'])
        session = requests.Session()
        session.auth = (PL_API_KEY, '')
        results = \
            session.get(
                ("https://api.planet.com/data/v1/item-types/" +
                 "{}/items/{}/assets/").format(i, item['id']))
       
        item_activation_url = results.json()[asset_type2]["_links"]["activate"]
       
        response = session.post(item_activation_url)
        print(response.status_code)
        while response.status_code != 204:
            time.sleep(30)
            response = session.post(item_activation_url)
            response.status_code = response.status_code
            print(response.status_code)
        
        item_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(i, item['id'])
        result = requests.get(item_url, auth=HTTPBasicAuth(PL_API_KEY, ''))
        download_url = result.json()[asset_type2]['location']  # KeyError: 'location'
        output_file = item['id'] + '.xml'

        with urllib.request.urlopen(download_url) as response, open(output_file, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)

PSScene4Band
20181217_025142_101e
204
20181217_025144_101e
204
20181217_025143_101e
204


In [38]:
os.getcwd()
os.mkdir('Original_Imagery')

In [39]:
sp = os.getcwd()
sf = os.listdir(sp)
dp = sp + ('\\Original_Imagery')
for file in sf:
    if file.endswith(('.tif','.xml')):
        shutil.move(os.path.join(sp,file), os.path.join(dp, file))

In [40]:
images = [f for f in os.listdir(dp) if os.path.isfile(os.path.join(dp, f))]

for image in images:
    folder_name = image.split('_')[0]

    new_path = os.path.join(dp, folder_name)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    old_image_path = os.path.join(dp, image)
    new_image_path = os.path.join(new_path, image)
    shutil.move(old_image_path, new_image_path)

In [41]:
folders = []
for r, d, f in os.walk(dp):
    for folder in d:
        folders.append(os.path.join(r, folder))

i = 0
for x in folders:
    files_to_mosaic = glob.glob(folders[i] + '/*.tif')
    files_string = " ".join(files_to_mosaic)
    command = "python " + sp + "\gdal_merge.py -o "+ files_string[-33:-25] + ".tif" + " -of gtiff " + files_string
    print(command)
    i = i + 1
    output = subprocess.getoutput(command)
    print(output)

python C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-3\PlanetCode-Integration-and-sorting-patch-3\gdal_merge.py -o 20181217.tif -of gtiff C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-3\PlanetCode-Integration-and-sorting-patch-3\Original_Imagery\20181217\20181217_025142_101e.tif C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-3\PlanetCode-Integration-and-sorting-patch-3\Original_Imagery\20181217\20181217_025143_101e.tif C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-3\PlanetCode-Integration-and-sorting-patch-3\Original_Imagery\20181217\20181217_025144_101e.tif
0...10...20...30...40...50...60...70...80...90...100 - done.


In [42]:
os.mkdir('Merged_Tifs')

In [43]:
sp2 = os.getcwd()
sf2 = os.listdir(sp2)
dp2 = sp2 + ('\\Merged_Tifs')
for file in sf2:
    if file.endswith(('.tif','.xml')):
        shutil.move(os.path.join(sp2,file), os.path.join(dp2, file))

In [44]:
os.chdir(sp2 + '\\Merged_Tifs')

In [45]:
files = []
for r, d, f in os.walk(dp2):
    for file in f:
        files.append(os.path.join(r, file))
        
timeavg = []
ndviavg = []

i = 0
for x in files:
    with rasterio.open(files[i]) as src:
            band_red = src.read(3)
    with rasterio.open(files[i]) as src:
            band_nir = src.read(4)       
            
    np.seterr(divide='ignore', invalid='ignore')
    ndvi = ((band_nir.astype(float) - band_red.astype(float)) / (band_nir.astype(float) + band_red.astype(float)))
    
    
    kwargs = src.meta
    kwargs.update(
    dtype = rasterio.float32,
    count = 1)
    with rasterio.open('ndvi_' + f[i], 'w', **kwargs) as dst:
        dst.write_band(1, ndvi.astype(rasterio.float32))
        
    ndvi2 = np.ma.masked_where(ndvi < 0., ndvi)
    
    fig2 = plt.figure(figsize=(10,10))
    ax = fig2.add_subplot(111)
    
    plt.title(f[i] + " NDVI Histogram", fontsize=18, fontweight='bold')
    plt.xlabel("NDVI values", fontsize=14)
    plt.ylabel("# pixels", fontsize=14)
    
    b = ndvi2[~np.isnan(ndvi2)]
    numBins = 20
    ax.hist(b,numBins,color='blue',alpha=0.8)
    ax.set_xlim([0, 1])
    fig2.savefig("ndvi_" + f[i][0:8] + "_histogram.png", dpi=200, bbox_inches='tight', pad_inches=0.7)

    
    ndvi_adjusted = ndvi * (-255.0)
    masked_ndvi_adjusted = np.ma.array(ndvi_adjusted, mask=np.isnan(ndvi_adjusted))
    cmap = plt.cm.RdYlGn
    cmap.set_bad(color = 'white',alpha = 0.0)
    img = masked_ndvi_adjusted.astype(np.uint8)
    plt.imsave("ndvi_" + f[i][0:8] + "_colormap.tiff", img, cmap=cmap)
    plt.close()
    
    avg = np.average(b)
    ndviavg.append(avg)
    time = f[i][4:8]
    timeavg.append(time)
       
    i = i + 1
    
fig3 = plt.figure(figsize=(10,10))
ax2 = fig3.add_subplot(111)
ax2.set_ylim([0, 1])
plt.plot(timeavg, ndviavg, color = 'blue', linewidth = 4)
plt.axhspan(0, 0.3, facecolor='r', alpha = 0.5, label = 'Below: Unhealthy')
plt.axhspan(0.3, 0.6, facecolor='y', alpha = 0.5, label = 'Below: Somewhat Healthy')
plt.axhspan(0.6, 1, facecolor='g', alpha = 0.5, label = 'Above: Healthy')
plt.legend()
plt.title("Average NDVI Over Time", fontsize=18, fontweight='bold')
plt.xlabel("Start Date & End Date", fontsize=14)
plt.ylabel("Average NDVI", fontsize=14)
fig3.savefig("AverageNDVI_OverTime.png", dpi=200, bbox_inches='tight', pad_inches=0.7)
plt.close()

In [46]:
sp3 = os.getcwd()
sf3 = os.listdir(sp3)

for file in sf3:
    if file.startswith('ndvi'):
        shutil.move(os.path.join(sp3,file), os.path.join(sp2,file))

In [47]:
os.chdir(sp2)
os.mkdir('Vegetation_Indices')

In [48]:
sp4 = os.getcwd()
sf4 = os.listdir(sp4)
dp4 = sp2 + '\\Vegetation_Indices'

for file in sf3:
    if file.startswith('ndvi'):
        shutil.move(os.path.join(sp4,file), os.path.join(dp4,file))

In [49]:
os.chdir('Merged_Tifs')

In [50]:
sp3 = os.getcwd()
sf3 = os.listdir(sp3)

for file in sf3:
    if file.startswith('Ave'):
        shutil.move(os.path.join(sp3,file), os.path.join(sp2,file))

In [51]:
os.chdir(sp2)
os.mkdir('Average_NDVI')

In [52]:
sp4 = os.getcwd()
sf4 = os.listdir(sp4)
dp4 = sp2 + '\\Average_NDVI'

for file in sf3:
    if file.startswith('Ave'):
        shutil.move(os.path.join(sp4,file), os.path.join(dp4,file))

In [53]:
os.chdir(sp2)

In [54]:
item_types = ['PSScene3Band']
asset_type = 'visual'

In [55]:
query = {
    "type": "AndFilter",
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [56]:
request = filters.build_search_request(
    query, item_types
)

In [57]:
dataset = client.quick_search(request)

stdout.write('id,cloud_cover,date\n')

id,cloud_cover,date


In [58]:
for item in dataset.items_iter(limit=None):
    props = item['properties']
    stdout.write('{0},{cloud_cover},{acquired}\n'.format(item['id'], **props))

20181217_025142_101e,0.01,2018-12-17T02:51:42.296592Z
20181217_025143_101e,0.01,2018-12-17T02:51:43.330487Z
20181217_025144_101e,0.01,2018-12-17T02:51:44.363859Z


In [59]:
for i in item_types:
    request = api.filters.build_search_request(query, [i])  # ,name = None, interval= 'day')
    dataset = client.quick_search(request)
    print(i)
    for item in dataset.items_iter(limit=100):
        print(item['id'])
        session = requests.Session()
        session.auth = (PL_API_KEY, '')
        results = \
            session.get(
                ("https://api.planet.com/data/v1/item-types/" +
                 "{}/items/{}/assets/").format(i, item['id']))
       
        item_activation_url = results.json()[asset_type]["_links"]["activate"]
       
        response = session.post(item_activation_url)
        print(response.status_code)
        while response.status_code != 204:
            time.sleep(30)
            response = session.post(item_activation_url)
            response.status_code = response.status_code
            print(response.status_code)
        
        item_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(i, item['id'])
        result = requests.get(item_url, auth=HTTPBasicAuth(PL_API_KEY, ''))
        download_url = result.json()[asset_type]['location']  # KeyError: 'location'
        output_file = item['id'] + '.tif'

        with urllib.request.urlopen(download_url) as response, open(output_file, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)

PSScene3Band
20181217_025142_101e
204
20181217_025143_101e
204
20181217_025144_101e
204


In [60]:
os.mkdir('Original_Ash_Imagery')

In [61]:
sp5 = os.getcwd()
sf5 = os.listdir(sp5)
dp5 = sp5 + ('\\Original_Ash_Imagery')
for file in sf5:
    if file.endswith('.tif'):
        shutil.move(os.path.join(sp5,file), os.path.join(dp5, file))

In [62]:
images = [f for f in os.listdir(dp5) if os.path.isfile(os.path.join(dp5, f))]

for image in images:
    folder_name = image.split('_')[0]

    new_path = os.path.join(dp5, folder_name)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    old_image_path = os.path.join(dp5, image)
    new_image_path = os.path.join(new_path, image)
    shutil.move(old_image_path, new_image_path)

In [63]:
folders = []
for r, d, f in os.walk(dp5):
    for folder in d:
        folders.append(os.path.join(r, folder))

i = 0
for x in folders:
    files_to_mosaic = glob.glob(folders[i] + '/*.tif')
    files_string = " ".join(files_to_mosaic)
    command = "python " + sp5 + "\gdal_merge.py -o "+ files_string[-33:-25] + ".tif" + " -of gtiff " + files_string
    print(command)
    i = i + 1
    output = subprocess.getoutput(command)
    print(output)

python C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-3\PlanetCode-Integration-and-sorting-patch-3\gdal_merge.py -o 20181217.tif -of gtiff C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-3\PlanetCode-Integration-and-sorting-patch-3\Original_Ash_Imagery\20181217\20181217_025142_101e.tif C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-3\PlanetCode-Integration-and-sorting-patch-3\Original_Ash_Imagery\20181217\20181217_025143_101e.tif C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-3\PlanetCode-Integration-and-sorting-patch-3\Original_Ash_Imagery\20181217\20181217_025144_101e.tif
0...10...20...30...40...50...60...70...80...90...100 - done.


In [64]:
os.mkdir('Merged_Ash_Tifs')

In [65]:
sp6 = os.getcwd()
sf6 = os.listdir(sp5)
dp6 = sp6 + ('\\Merged_Ash_Tifs')
for file in sf6:
    if file.endswith('.tif'):
        shutil.move(os.path.join(sp6,file), os.path.join(dp6, file))

In [66]:
os.chdir(dp6)

In [67]:
flags = [i for i in dir(cv2) if i.startswith('COLOR')]

In [68]:
range1_lo = (130, 130, 89, 255)
range1_hi = (166, 152, 129, 255)

range2_lo = (153, 146, 137, 255)
range2_hi = (219, 208, 195, 255)

range3_lo = (120, 119, 114, 255)
range3_hi = (125, 126, 121, 255)

range4_lo = (144, 144, 129, 255)
range4_hi = (150, 150, 137, 255)

In [69]:
files = []
for r, d, f in os.walk(dp6):
    for file in f:
        files.append(os.path.join(r, file))

ash_pixels = []
timeavg = []

i = 0
for x in files:
    img = cv2.imread(files[i])
    img2 = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)
    
    mask_range1 = cv2.inRange(img2, range1_lo, range1_hi)
    mask_range2 = cv2.inRange(img2, range2_lo, range2_hi)
    mask_range3 = cv2.inRange(img2, range3_lo, range3_hi)
    mask_range4 = cv2.inRange(img2, range4_lo, range4_hi)
    
    ash_mask = mask_range1+mask_range2+mask_range3+mask_range4
    ash_result = cv2.bitwise_and(img2, img2, mask=ash_mask)
    
    img[ash_mask>0]=(0,0,255)
    cv2.imwrite('Ash_Overlay_' + f[i] + '.tif', img)
    
    pixel_count = cv2.countNonZero(ash_mask)
    ash_pixels.append(pixel_count)
    
    time = f[i][4:8]
    timeavg.append(time)
    
    i = i + 1
    
fig3 = plt.figure(figsize=(10,10))
ax2 = fig3.add_subplot(111)
plt.plot(timeavg, ash_pixels, color = 'blue', linewidth = 4)
plt.title("Ash Coverage Over Time", fontsize=18, fontweight='bold')
plt.xlabel("Start Date & End Date", fontsize=14)
plt.ylabel("# of Pixels", fontsize=14)
fig3.savefig("Ash_Coverage_OverTime.png", dpi=200, bbox_inches='tight', pad_inches=0.7)
plt.close()

In [70]:
sp7 = os.getcwd()
sf7 = os.listdir(sp7)

for file in sf7:
    if file.startswith('Ash'):
        shutil.move(os.path.join(sp7,file), os.path.join(sp2,file))

In [71]:
os.chdir(sp2)
os.mkdir('Segmentation_Ash')

In [72]:
sp8 = os.getcwd()
sf8 = os.listdir(sp8)
dp8 = sp2 + '\\Segmentation_Ash'

for file in sf8:
    if file.startswith('Ash'):
        shutil.move(os.path.join(sp8,file), os.path.join(dp8,file))

In [73]:
newfldr = input('Would you like to make a new folder to store data? y/n: ')

if newfldr == 'n':
    print('Goodbye')
elif newfldr == 'y':
    nf = os.mkdir(input('Enter new folder name: '))

Would you like to make a new folder to store data? y/n: y
Enter new folder name: Poop


In [82]:
sp9 = os.getcwd()
sf9 = os.listdir(sp9)
dp9 = sp2 + nf

for folder in sf9:
    if folder.startswith(('Ave', 'Mer', 'Ori', 'Seg', 'Veg')):
        shutil.move(os.path.join(sp9,folder), os.path.join(dp9, folder))

In [88]:
move = input('Would you like to move your new folder to Desktop? y/n: ')

if move == 'n':
    print('Folder Not Moved')
elif move == 'y':
    sp10 = os.getcwd()
    sf10 = os.listdir(sp10)
    dp10 = '\\Desktop'
    shutil.move(os.path.join(sp10,nf), os.path.join(dp10, nf))
    print('Folder Moved')

Would you like to move your new folder to Desktop? y/n: y
Folder Moved


In [4]:
restart = input('Would you like to run another dataset? y/n: ')

if restart == 'n':
    print('Have fun')
    pass
elif restart == 'y':
    os.execl(sys.executable, '"{}"'.format(sys.executable), *sys.argv)

Would you like to run another dataset? y/n: n
Have fun
